In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from google.oauth2 import service_account
from langchain_google_community.gcs_file import GCSFileLoader
from langchain.document_loaders import JSONLoader
import os

In [4]:

credentials = service_account.Credentials.from_service_account_file('credentials.json')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

In [13]:
def load_json(file_path, jq_schema=None):
    return JSONLoader(file_path, jq_schema=jq_schema, text_content=False)

In [14]:
loader = GCSFileLoader(project_name=credentials.project_id,
                       bucket="kennedyskis",
                       blob="1st-nixon-kennedy-debate-19600926.json",
                       loader_func=load_json )

In [1]:
loader = load_json("yuma-az-19600409.json", ".")

In [8]:
documents = loader.load()

In [10]:
documents[0]

Document(metadata={'source': '/Users/mikeharmon/Desktop/LLMs/rag-jfk/yuma-az-19600409.json', 'seq_num': 1}, page_content='')

In [10]:
print(documents[0].page_content)

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [36]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="llama3:latest"
)

In [37]:
from langchain_chroma.vectorstores import Chroma

In [38]:
vector_store = Chroma(
    collection_name="first-debate",
    embedding_function=embedding,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [51]:
vectordb = vector_store.from_documents(documents=texts,
                        embedding=embedding,
                        persist_directory="data",
                        collection_name="first-debate")

In [65]:
results = vectordb.similarity_search_by_vector(
    embedding=embedding.embed_query("What did Kennedy think about Lincoln?"), k=5
)

In [66]:
results

[Document(metadata={'source': 'gs://harmon-kennedy/1st-nixon-kennedy-debate-19600926.txt'}, page_content="And if we meet our responsibilities, I think freedom will conquer. If we fail--if we fail to move ahead, if we fail to develop sufficient military and economic and social strength here in this country, then I think that the tide could begin to run against us, and I don't want historians 10 years from now, to say, these were the years when the tide ran out for the United States. I want them to say, these were the years when the tide came in, these were the years when the United States started to move again. That's the question before the American people, and only you can decide what you want, what you want this country to be, what you want to do with the future.\nI think we're ready to move. And it is to that great task, if we are successful, that we will address ourselves.\nMR. SMITH: Thank you very much, gentlemen.\nThis hour has gone by all too quickly. Thank you very much for pe

In [67]:
from langchain_ollama import ChatOllama

from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [68]:
template = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/mikeharmon/miniconda3/envs/llm_env/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [79]:
llm = ChatOllama(model="llama3", temperature=0)

In [80]:
stuff_documents_chain = create_stuff_documents_chain(llm, template)

In [81]:
rag_chain = create_retrieval_chain(
                    vectordb.as_retriever(), 
                    stuff_documents_chain
)

In [82]:
response = rag_chain.invoke({"input": "How Did Kennedy feel about Lincoln?"})

In [84]:
response["answer"]

"The context does not mention John F. Kennedy's feelings about Abraham Lincoln. It appears to be a transcript of a speech or debate between two politicians, likely Richard Nixon and John F. Kennedy, during the 1960 presidential election. There is no mention of Lincoln in this specific passage."